# Capítulo 5 — O Desempenho de Sistemas de Controle com Realimentação

**Aluno(a):** Geovana Stefani Lopes Bezerra  
**Disciplina:** Controle e Automação  
**Professor:** Moacy Pereira da Silva  
**Semestre:** 2025.2  

Este notebook faz parte do estudo dirigido da disciplina de Controle e Automação com foco em aplicações de Indústria 4.0.  
O capítulo 5 do livro *Sistemas de Controle Moderno* (Dorf & Bishop) apresenta os critérios e métricas utilizadas para avaliar o desempenho de sistemas de controle em laço fechado.


## 1. Introdução

O desempenho de um sistema de controle em malha fechada é avaliado por como o sistema responde a entradas de referência e a perturbações. O Capítulo 5 de Dorf & Bishop organiza essa avaliação em torno de métricas quantitativas (tempo de subida, sobressinal, tempo de acomodação, erro em regime permanente, etc.) e procedimentos analíticos que relacionam essas métricas com as características do sistema em malha aberta (polos, zeros, ganho estático e margens de estabilidade). Neste notebook adotamos a mesma organização: primeiro apresentamos os conceitos, depois as fórmulas úteis e, por fim, exemplificamos com simulações que ilustram os trade-offs clássicos entre desempenho e robustez.

## 2. Fundamentos teóricos 

### 2.1 Resposta transitória e resposta em regime permanente
Dorf & Bishop distinguem claramente a resposta transitória (comportamento logo após uma mudança de referência ou perturbação) da resposta em regime permanente (comportamento quando o efeito transiente já decaiu). A resposta transitória é caracterizada por parâmetros como tempo de subida (Tr), tempo até o pico (Tp), sobressinal (Mp) e tempo de acomodação (Ts). A resposta em regime permanente é medida pelo erro em regime (ess) em relação à referência desejada. Esta distinção orienta o projeto de controladores porque medidas que melhoram a resposta transitória (por exemplo, aumentar ganho) podem degradar o comportamento em regime ou reduzir a robustez.

### 2.2 Modelos e modos dominantes
Para sistemas lineares de ordem superior, a noção de modos dominantes é utilizada para reduzir a análise transitória a uma forma similar à de um sistema de segunda ordem. Polos complexos conjugados próximos do eixo imaginário dominam a resposta transitória oscilatória; polos reais lentos dominam a resposta lenta de primeira ordem. Dorf & Bishop mostram como, em muitos projetos, é aceitável aproximar a dinâmica pelos modos dominantes para estimar Tr, Mp e Ts.

### 2.3 Erro estacionário, tipo do sistema e constantes de erro
Defina a função de malha aberta L(s)=C(s)G(s). O tipo do sistema é o número de polos em s=0 na função L(s). A capacidade do sistema em seguir referências polinomiais depende do tipo: tipo 0 (erro finito para degrau), tipo 1 (erro zero para degrau, erro finito para rampa), tipo 2 (erro zero para rampa), e assim por diante. Dorf & Bishop tratam as constantes de erro (Kp, Kv, Ka) e relacionam-nas com ess para referências específicas.

### 2.4 Compensadores e efeitos no desempenho
Os efeitos dos compensadores clássicos (P, PI, PD, PID, lead/lag) são discutidos em termos de suas ações sobre polos, zeros e ganho: PI reduz ess para degrau adicionando um polo em zero (integração); PD adiciona fase (melhora Tr/Mp); lead/lag são usados para moldar margem de fase e ganho, buscando compromisso entre desempenho e robustez. O capítulo enfatiza a necessidade de avaliar margens (ganho/fase) para garantir robustez contra incertezas de modelo.

## 3. Métricas de desempenho — definições e relações matemáticas

### 3.1 Definições formais
- **Sobressinal (Mp)**: porcentagem que a resposta ultrapassa o valor final, Mp = (y_max - y_final)/y_final × 100%.  
- **Tempo de subida (Tr)**: tempo para a resposta ir de 10% a 90% do valor final (convenção comum).  
- **Tempo até o pico (Tp)**: tempo onde ocorre o máximo (y_max).  
- **Tempo de acomodação (Ts)**: tempo após o qual a resposta permanece dentro de uma banda ±δ (tipicamente δ=2% ou 5%) do valor final.  
- **Erro em regime permanente (ess)**: lim_{t→∞} (r(t) − y(t)).  

### 3.2 Fórmulas para o segundo-ordem padrão
Para um sistema de segunda ordem padrão G(s)=ω_n^2/(s^2 + 2ζω_n s + ω_n^2):  
- Mp = exp(−ζπ/√(1−ζ^2))  (fração).  
- Tp = π/(ω_n √(1−ζ^2)).  
- Tr ≈ 1.8/ω_n (apróx., para ζ ≈ 0.2−0.8).  
- Ts (±2%) ≈ 4/(ζ ω_n).  

Essas relações são úteis porque permitem projetar especificações de desempenho em termos de ζ e ω_n, que por sua vez orientam o posicionamento de polos desejados em projeto por lugar das raízes ou sendo alvo em técnicas de alocação de polos.

### 3.3 Relação com margens de estabilidade
Dorf & Bishop destacam que desempenho e robustez competem: aumentar ganho pode reduzir ess e acelerar resposta, mas diminui margem de fase e ganho, aproximando o sistema da instabilidade. Portanto, o projeto frequentemente envolve concessões e verificação por margens e diagramas de Bode.

In [ ]:
# !pip install control --quiet

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from control import tf, feedback, step_response, bode, margin
plt.rcParams['figure.figsize'] = (10,4)

def performance_metrics(t, y, reference=1.0, settling_band=0.02):
    """Calcula Mp, Tr (10-90%), Tp, Ts (banda), ess a partir de amostras t,y."""
    t = np.asarray(t)
    y = np.asarray(y)
    final = y[-1]
    y_max = np.max(y)
    Mp = 0.0
    if final != 0:
        Mp = (y_max - final) / abs(final) * 100.0
    # Tp
    Tp = t[np.argmax(y)]
    # Rise time 10-90%
    try:
        t10 = t[np.where(y >= 0.1*final)[0][0]]
        t90 = t[np.where(y >= 0.9*final)[0][0]]
        Tr = t90 - t10
    except Exception:
        Tr = float('nan')
    # Settling time (first time it enters and stays within band)
    band = settling_band * abs(final)
    Ts = float('nan')
    idxs = np.where(np.abs(y - final) <= band)[0]
    if idxs.size > 0:
        for i in idxs:
            if np.all(np.abs(y[i:] - final) <= band):
                Ts = t[i]
                break
    ess = reference - final
    return {'Mp_percent': Mp, 'Tr_s': Tr, 'Tp_s': Tp, 'Ts_s': Ts, 'ess': ess}


## 4. Plantas de exemplo (definição física e modelagem)

Serão utilizadas três plantas clássicas para ilustrar conceitos do capítulo:
- **Planta A (processo com integrador e modos lentos):** G_A(s) = 1 / (s (s+1) (s+2)). Representa um processo com integrador — útil para discutir erro estacionário e necessidade de ação integral.  
- **Planta B (tanque, 1ª ordem lenta):** G_B(s) = K/(τ s + 1), com K=1, τ=5 s. Modelo típico de nível de tanque.  
- **Planta C (motor DC simplificado, 1ª ordem rápida):** G_C(s) = K/(τ s + 1), com K=0.5, τ=0.5 s. Modelo típico de atuador rápido.

Cada planta será testada com três controladores: P, PI e PID (formas clássicas), usando parâmetros iniciais que evidenciem os trade-offs. Em seguida calcularemos métricas e discutiremos.

In [ ]:
# Definição das plantas
s = tf([1,0],[1])
G_A = 1/(s*(s+1)*(s+2))
K_B = 1.0; tau_B = 5.0
G_B = tf([K_B],[tau_B, 1])
K_C = 0.5; tau_C = 0.5
G_C = tf([K_C],[tau_C, 1])
G_A, G_B, G_C


## 5. Simulações: controladores, parâmetros e execução

Parâmetros iniciais escolhidos para demonstração (pode-se ajustar em varredura):
- Kp = 5.0, Ki = 1.0, Kd = 0.1  
- Controladores implementados na forma de funções de transferência:  
  - P(s) = Kp  
  - PI(s) = Kp + Ki/s  (implementado como (Kp*s + Ki)/s)  
  - PID(s) = Kp + Ki/s + Kd*s  (implementado em forma canônica)

A seguir realizamos as simulações (resposta ao degrau unitário na malha fechada) e extraímos métricas.

In [ ]:
# Controladores
Kp = 5.0; Ki = 1.0; Kd = 0.1
C_P = tf([Kp],[1])
C_PI = tf([Kp, Ki],[1, 0])   # (Kp*s + Ki)/s
C_PID = tf([Kd, Kp, Ki],[1, 0])

def simulate_and_report(G, C, name, t_final=60, ref=1.0):
    L = C*G
    T = feedback(L, 1)
    t = np.linspace(0, t_final, 2001)
    t_sim, y = step_response(T, T=t)
    metrics = performance_metrics(t_sim, y, reference=ref)
    return {'t': t_sim, 'y': y, 'metrics': metrics, 'T': T}

# Simulações para cada planta e controlador
results = {}
for name, G_var, tfinal in [('Planta A', G_A, 60), ('Planta B', G_B, 100), ('Planta C', G_C, 10)]:
    results[name] = {}
    for C, cname in [(C_P, 'P'), (C_PI, 'PI'), (C_PID, 'PID')]:
        r = simulate_and_report(G_var, C, cname, t_final=tfinal)
        results[name][cname] = r

# Plot comparativo para cada planta
for name in results:
    plt.figure()
    for cname in results[name]:
        d = results[name][cname]
        plt.plot(d['t'], d['y'], label=f"{cname} (Mp={d['metrics']['Mp_percent']:.1f}%, Ts={d['metrics']['Ts_s']})")
    plt.title(f'Resposta ao degrau - {name}')
    plt.xlabel('Tempo (s)')
    plt.ylabel('Saída')
    plt.grid(True)
    plt.legend()
    plt.show()


## 6. Discussão detalhada das simulações (interpretação e ligação com teoria)

Nesta seção interpretamos, de forma aprofundada, os resultados obtidos nas simulações à luz dos conceitos apresentados no Capítulo 5 de Dorf & Bishop.

- **Planta A (G_A = 1/[s(s+1)(s+2)])**: por conter um integrador em malha aberta, o sistema tende a possuir comportamento de tipo 1 quando controller não remove integrador. A ação P aumenta ganho e pode acelerar resposta, mas não elimina o erro em regime para tipos inferiores — contudo, neste caso o integrador presente faz com que o erro para degrau possa ser reduzido naturalmente. O controlador PI elimina o erro em regime (para degrau) por integrar o erro, mas acrescenta dinâmica que pode reduzir a margem de fase e aumentar o sobressinal. O PID, se bem ajustado, tenta combinar redução de erro com amortecimento do transiente (via termo D). Assim, espera-se observar nas curvas: PI com menor ess, PID com menor Mp comparado a PI mal sintonizado.

- **Planta B (tanque, G_B)**: sendo uma planta lenta (τ=5s), observa-se que ganhos elevados geram respostas mais rápidas porém podem provocar sobressinal se houver falta de fase suficiente. Em aplicações de nível, costuma-se preferir PI com ajuste conservador para eliminar erro sem induzir oscilações perigosas.

- **Planta C (motor, G_C)**: planta rápida, portanto P com ganho moderado pode ser suficiente; o termo D tende a melhorar amortecimento sem aumentar o estado estacionário. Espera-se Tr pequeno e Ts também pequeno; overshoot depende do ajuste de Kp/Kd.

Além dessas observações específicas, ressalta-se os seguintes pontos gerais:
1. **Trade-off desempenho × robustez**: conforme indicado no capítulo, melhorias de desempenho (reduzir Ts, Tr) costumam reduzir margens de estabilidade. É necessário confirmar margens via análise de Bode/margin.
2. **Importância da modelagem**: variâncias de τ, ganhos ou atrasos não modelados podem tornar um controlador projetado por ajuste manual instável em campo. Dorf & Bishop enfatizam validação experimental e análise de sensibilidade.
3. **Anti-windup e saturação**: em plantas com atuadores saturáveis, o termo integrativo requer mecanismos anti-windup para evitar comportamento indevido — detalhe prático tratado no capítulo e que deve ser considerado em implementações reais.


In [ ]:
# Cálculo de margens (exemplo na Planta A com PID)
L = C_PID * G_A
gm, pm, sm, wg, wp, ws = margin(L)
print('Gain margin (factor):', gm)
print('Phase margin (deg):', pm)
print('Gain crossover freq (rad/s):', wg)
print('Phase crossover freq (rad/s):', wp)


## 7. Relação com Indústria 4.0 (APC, IIoT, PIMS, Digital Twin)

Este capítulo do livro fornece a base para avaliar desempenho no nível de controlador, nos ambientes de Indústria 4.0 essa avaliação é integrada a camadas superiores: monitoramento, otimização e gêmeos digitais.

- **APC (Advanced Process Control)**: métricas de desempenho (ess, Ts, Mp) são usadas como critérios para decidir quando migrar de controladores PID para estratégias baseadas em modelo (MPC). Em malhas com muitas interações ou restrições de processo, o MPC pode alcançar desempenho superior respeitando limites operacionais.

- **IIoT / PIMS (PI System, Historian)**: a telemetria de processo pode armazenar métricas de desempenho continuamente (por exemplo, calcular Ts e Mp em janelas deslizantes) para detectar degradação de performance, disparar retuning automático ou alertas de manutenção.

- **Digital Twin**: ao integrar os modelos das plantas e do controlador em um gêmeo digital, é possível simular cenários (mudança de processo, falhas, variação de parâmetros) e avaliar o impacto sobre as métricas de desempenho antes de aplicar no campo.
